In [1]:
import json
with open('summary-meta copy') as f:
    line = f.readline()

category_topic_list = list(json.loads(line))
#for content in category_topic_list:
 #   print(content)
print(category_topic_list)    


[{u'Category': u'smrt corporate', u'Topic': u'corporate & financials', u'Tonality': u'neutral'}, {u'Category': u'smrt buses', u'Topic': u'facilities & services', u'Tonality': u'neutral'}, {u'Category': u'smrt trains', u'Topic': u'rail & engineering', u'Tonality': u'neutral'}, {u'Category': u'unknown', u'Topic': u'unknown', u'Tonality': u'unknown'}, {u'Category': u'smrt buses', u'Topic': u'service announcements', u'Tonality': u'positive'}, {u'Category': u'smrt corporate', u'Topic': u'regulations & ops', u'Tonality': u'neutral'}, {u'Category': u'smrt trains', u'Topic': u'crisis / delays & distributions / accidents / safety', u'Tonality': u'neutral'}, {u'Category': u'smrt corporate', u'Topic': u'financials', u'Tonality': u'neutral'}, {u'Category': u'smrt corporate', u'Topic': u'rail & engineering', u'Tonality': u'neutral'}, {u'Category': u'smrt trains', u'Topic': u'rail & engineering', u'Tonality': u'neutral'}, {u'Category': u'smrt trains', u'Topic': u'rail & engineering', u'Tonality': u'

In [2]:
print(category_topic_list[3])
with open('summary-docs') as f:
    contentLines = f.readlines()

contentLines = [ line.strip() for line in contentLines]


{u'Category': u'unknown', u'Topic': u'unknown', u'Tonality': u'unknown'}


In [3]:
len(contentLines)
print(category_topic_list[0])

{u'Category': u'smrt corporate', u'Topic': u'corporate & financials', u'Tonality': u'neutral'}


In [4]:

contentLines = [ line.lower() for line in contentLines]

In [5]:
contentLines[2]

"strong signals have emerged that the major rail reform about smrt corp is imminent. if market wags are to be believed, the new rail financing framework which will relieve the transport operator of excruciating capital expenditure (capex) burden and free up its cash flow could happen this financial year, contrary to analysts' general expectation for reforms in 2017 or beyond. in addition, smrt's stellar showing despite the street's negative take on the stock denotes a sooner-than-later expectation. under the new framework, temasek-owned smrt is expected to emerge as an asset-light company and a pure operator while the government would own all operating assets and the associated risks for major capital asset investments. these assets will then be leased to smrt. finally, analysts expect an announcement on rail reform to be a clear catalyst for smrt shares, which jumped 60 per cent last year following news of the bus reform."

In [8]:
###TBC preprocess 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


def preprocessing(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #remove stop words
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]
    
    ##lower case
    tokens = [token.lower() for token in tokens]
    
    ##lemma
    lmtzr = WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)
    return (text)
    
print(preprocessing('she is good better running apples ran car cars automobiles pretty beautyful beautyfully children'))
#＃contentLines = map(lambda x: preprocessing(x) ,contentLines)
    

good better running apple ran car car automobile pretty beautyful beautyfully child


In [67]:
##contentLines = map(lambda x: x.split('.')[0] ,contentLines)
contentLinesTmp = []
category_topic_list_tmp = []
import random

for i in range(0,len(contentLines)):
    ##print(category_topic_list[i])
    #if(category_topic_list[i].get('Category') == 'smrt bus' or category_topic_list[i].get('Category') == u'smrt trains'):
    #    print(category_topic_list[i])
    #    if(random.randint(0,4)!=4):
    #        print("filter!!!!")
    #        continue            
    splited_sentence = contentLines[i].split('.')
    ###contentLinesTmp.append(contentLines[i])
    j = 0
    while j<len(splited_sentence):
        if(j+1<len(splited_sentence)):
            contentLinesTmp.append(splited_sentence[j] + ' ' + splited_sentence[j+1])             
        else:
            contentLinesTmp.append(splited_sentence[j] ) 
        category_topic_list_tmp.extend([category_topic_list[i]])
        j +=2

In [7]:
contentLinesTmp[30:50]

NameError: name 'contentLinesTmp' is not defined

In [8]:
print(len(contentLinesTmp))
print(len(category_topic_list_tmp))

category_topic_list = category_topic_list_tmp
contentLines = contentLinesTmp

NameError: name 'contentLinesTmp' is not defined

In [9]:
print(contentLines[0:10])

['', u"route smrt bus service 75 167 amended sunday ( may 1 ) , anderson bridge direction st andrew 's road closed bus heavy vehicle . route service amended ply esplanade drive instead .", u"strong signal emerged major rail reform smrt corp imminent . market wag believed , new rail financing framework relieve transport operator excruciating capital expenditure ( capex ) burden free cash flow could happen financial year , contrary analyst ' general expectation reform 2017 beyond . addition , smrt 's stellar showing despite street 's negative take stock denotes sooner-than-later expectation . new framework , temasek-owned smrt expected emerge asset-light company pure operator government would operating asset associated risk major capital asset investment . asset leased smrt . finally , analyst expect announcement rail reform clear catalyst smrt share , jumped 60 per cent last year following news bus reform .", u"forum writer mr han jiajing ( `` shorten interval time peak-hour train '' ; 

In [15]:
###sklearn CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1,2),min_df=2,stop_words='english',norm='l2') ##用三过于稀疏了 只有0.65准确率在验证ji
###best: 用ngrame_range=(1,2) 不做任何特征处理时候validation accuracy 0.780 training accuracy 0.998 特征维度 (1766, 77971)
ngram_content = cv.fit_transform(contentLines).toarray()
word_dict = cv.get_feature_names()
print(ngram_content.shape)

(817, 6915)


In [16]:
category_text_list = category_topic_list
print(len(category_text_list))
###filter category:smrt buses, smrt corporate, competitors,  smrt taxis,smrt trains 'service excellence', 'customer satisfaction',
###'delay/disruption' 'unknown',
category_text_list_tmp  = []
contentLines_tmp = []
category_topic_list_tmp = []
for i in range(0,len(category_topic_list)):
    if(category_topic_list[i].get('Topic') in [
                                               'crisis / delays & distributions / accidents / safety', 
                                               'general / others','facilities & services',
                                              'service announcements','service excellence']):
        category_text_list_tmp.append(category_topic_list[i].get('Category'))
        contentLines_tmp.append(contentLines[i])
        category_topic_list_tmp.append(category_topic_list[i])

print(len(contentLines_tmp))
print(len(category_text_list_tmp))

category_text_list = category_text_list_tmp
contentLines = contentLines_tmp
category_topic_list = category_topic_list_tmp

print(len(category_text_list))
print(len(contentLines))
print(len(category_topic_list))   

817
817
817
817
817
817


In [17]:
##转化成category id list 用dict存name to id mapping
category_text_list = [line['Topic'] for line in category_topic_list]
category_text_set = set(category_text_list)
categort2id_dict = dict(zip(category_text_set,range(len(category_text_set))))
category_id_list = map(lambda x: categort2id_dict[x] ,category_text_list)
print(len(category_id_list))
print(len(contentLines))

###看一下分类数量
arr_appear = dict((a, category_text_list.count(a)) for a in category_text_list);
print(arr_appear)
class_weight = dict((a, category_id_list.count(a)) for a in category_id_list);
print(class_weight)



print(len(contentLines))
print(len(category_id_list))

print(contentLines[0:10])

817
817
{u'service announcements': 40, u'crisis / delays & distributions / accidents / safety': 323, u'facilities & services': 264, u'service excellence': 37, u'general / others': 153}
{0: 40, 1: 323, 2: 264, 3: 37, 4: 153}
817
817
[u"route smrt bus service 75 167 amended sunday ( may 1 ) , anderson bridge direction st andrew 's road closed bus heavy vehicle . route service amended ply esplanade drive instead .", u'june 14 , smrt adding seven new wheelchair accessible bus ( wab ) service , thereby increasing wab service route 52 total . additional bus service added wab service 106 , 190 , 301 , 804 , 911 , 963 983 .', u'reader , liu jiakeng , shared compliment visiting malaysian friend towards local efficient public transport network . however , reader claimed evident last breakdown smrt lta well prepared towards new breakdown . first , cited missing notification service disruption lta app . second , claimed shuttle bus came limited destination able accommodate passenger . third , high

In [14]:
categort2id_dict

{u'crisis / delays & distributions / accidents / safety': 1,
 u'facilities & services': 2,
 u'general / others': 4,
 u'service announcements': 0,
 u'service excellence': 3}

In [121]:
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score
import numpy as np
import pandas as pd
from pandas import DataFrame

In [80]:

###Optional: elect feature
from sklearn.feature_selection import RFE
print "feature selection"    
##for train, test in cv:
svc = linear_model.LogisticRegression(C=5)
selector = RFE(svc, 10000, step=0.1)
features = selector.fit_transform(ngram_content, category_id_list)
print "feature selection finshed"

feature selection


ValueError: Found input variables with inconsistent numbers of samples: [5187, 1993]

In [86]:
##train using SVM: TB Tuned
 
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(ngram_content,np.array(category_id_list),test_size=0.1)



In [122]:

 
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(ngram_content,np.array(category_id_list),test_size=0.1)
svm = SVC(kernel=str('linear'))
print("start","train")
svm.fit(X_train, y_train)
print("finish","train")

('start', 'train')
('finish', 'train')


In [123]:
test_result = svm.predict(X_test)
print(len(test_result))
results =  (test_result == y_test) 
score = [1 for i in results if i]
accuracy = float(sum(score))/float(len(results))
print('validation accuracy %.3f' %accuracy )
print('training accuracy %.3f' %svm.score(X_train,y_train ) )

82
validation accuracy 0.805
training accuracy 0.977


In [33]:
##各种model来一遍

In [19]:
from sklearn import linear_model
import numpy as np
print "logistic回归"    
for train, test in cv:
    svc = linear_model.LogisticRegression(C=5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False).fit(ngram_content[train], np.array(category_id_list)[train])    
    ###linear_model.LogisticRegression(C=5,class_weight='balanced').fit(ngram_content[train], np.array(category_id_list)[train])    
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        svc.score(ngram_content[train], np.array(category_id_list)[train]), svc.score(ngram_content[test], np.array(category_id_list)[test])))


logistic回归
train score: 0.992, test score: 0.768

train score: 0.993, test score: 0.805

train score: 0.992, test score: 0.780

train score: 0.992, test score: 0.829

train score: 0.990, test score: 0.817

train score: 0.993, test score: 0.793

train score: 0.992, test score: 0.841

train score: 0.990, test score: 0.768

train score: 0.990, test score: 0.878

train score: 0.992, test score: 0.793

train score: 0.996, test score: 0.756

train score: 0.993, test score: 0.780

train score: 0.990, test score: 0.841

train score: 0.990, test score: 0.841

train score: 0.993, test score: 0.756

train score: 0.992, test score: 0.793

train score: 0.990, test score: 0.768

train score: 0.992, test score: 0.744

train score: 0.992, test score: 0.768

train score: 0.992, test score: 0.744



In [18]:
# 总得切分一下数据咯（训练集和测试集）
from pandas import DataFrame
from sklearn import cross_validation
df_train_data = DataFrame()
cv = cross_validation.ShuffleSplit(len(ngram_content), n_iter=20, test_size=0.1,
    random_state=0)

# 各种模型来一圈


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [104]:
{u'service announcements': 40,
 u'crisis / delays & distributions / accidents / safety': 323,
 u'facilities & services': 264,
 u'service excellence': 37,
 u'general / others': 153}

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [38]:
a = svc.coef_[0].argsort()[-20:][::-1]

In [39]:
for a1 in a:
    print(word_dict[a1])

service
bus service
sept
service extended
road
16
polling
depart
cd
extended eve
190
11pm
bsep
parking
run
service bus
resident
admiralty station
orchard road
till


In [58]:
###gridsearch using logistic regression

tuned_parameters = [{'C':[1,5,10],'penalty':["l1",'l2']}]   
    
scores = ['r2']

for score in scores:
    
    print score
    
    clf = GridSearchCV(linear_model.LogisticRegression(class_weight='balanced'), tuned_parameters, cv=5)
    clf.fit(ngram_content, np.array(category_id_list))

    print("别！喝！咖！啡！了！最佳参数找到了亲！！：")
    print(clf.best_score_)
    print ""
    #best_estimator_ returns the best estimator chosen by the search
    print(clf.best_estimator_)
    print ""
    print("得分分别是:")
    print ""
    #grid_scores_的返回值:
    #    * a dict of parameter settings
    #    * the mean score over the cross-validation folds 
    #    * the list of scores for each fold
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print ""


r2


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


别！喝！咖！啡！了！最佳参数找到了亲！！：
0.814835787089

LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

得分分别是:

0.712 (+/-0.009) for {'penalty': 'l1', 'C': 1}
0.800 (+/-0.005) for {'penalty': 'l2', 'C': 1}
0.772 (+/-0.004) for {'penalty': 'l1', 'C': 5}
0.811 (+/-0.006) for {'penalty': 'l2', 'C': 5}
0.775 (+/-0.007) for {'penalty': 'l1', 'C': 10}
0.815 (+/-0.006) for {'penalty': 'l2', 'C': 10}



In [11]:
   
print "随机森林回归/Random Forest(n_estimators = 100)"    
for train, test in cv:    
    
    svc = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=0.48979591836734693, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False, n_estimators = 100).fit(ngram_content[train], np.array(category_id_list)[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(svc.score(ngram_content[train], np.array(category_id_list)[train]), svc.score(ngram_content[test], np.array(category_id_list)[test])))


随机森林回归/Random Forest(n_estimators = 100)


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.904, test score: 0.757



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.906, test score: 0.774



...

train score: 0.905, test score: 0.751



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.901, test score: 0.718



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.902, test score: 0.734



In [ ]:
###gridsearch using rf

##tuned_parameters = [{'n_estimators':[100],'max_depth':[5,10,50,None],'min_impurity_split':np.linspace(0,1,50),
##                    }]  

tuned_parameters=[{'criterion':['entropy'],'min_impurity_split':np.linspace(0,1,50)},
                 {'criterion':['gini'],'min_impurity_split':np.linspace(0,0.5,50)},
                  {'max_depth':range(2,10)},
                  {'min_samples_split':range(2,30,2)}]
    
scores = ['r2']

for score in scores:
    
    print score
    
    clf = GridSearchCV(RandomForestClassifier(n_estimators=10,class_weight='balanced'), tuned_parameters, cv=5)
    clf.fit(ngram_content, np.array(category_id_list))

    print("别！喝！咖！啡！了！最佳参数找到了亲！！：")
    print ""
    #best_estimator_ returns the best estimator chosen by the search
    print(clf.best_estimator_)
    print ""
    print("得分分别是:")
    print ""
    #grid_scores_的返回值:
    #    * a dict of parameter settings
    #    * the mean score over the cross-validation folds 
    #    * the list of scores for each fold
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print ""


r2


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.


别！喝！咖！啡！了！最佳参数找到了亲！！：

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=0.48979591836734693, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

得分分别是:

0.730 (+/-0.008) for {'min_impurity_split': 0.0, 'criterion': 'entropy'}
0.738 (+/-0.010) for {'min_impurity_split': 0.020408163265306121, 'criterion': 'entropy'}
0.712 (+/-0.003) for {'min_impurity_split': 0.040816326530612242, 'criterion': 'entropy'}
0.732 (+/-0.008) for {'min_impurity_split': 0.061224489795918366, 'criterion': 'entropy'}
0.717 (+/-0.010) for {'min_impurity_split': 0.081632653061224483, 'criterion': 'entropy'}
0.717 (+/-0.009) for {'min_impurity_split': 0.1020408163265306, 'criterion': 'entropy'}
0.725

In [127]:
print "支持向量回归/SVC linear 最好适用于多维度小数据m》n 否则用高斯rbf "

for train, test in cv:
    svc = svm.SVC(kernel=str('linear')).fit(ngram_content[train], np.array(category_id_list)[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(svc.score(ngram_content[train], np.array(category_id_list)[train]), svc.score(ngram_content[test], np.array(category_id_list)[test])))
##

支持向量回归/SVC linear 最好适用于多维度小数据m》n 否则用高斯rbf 


AttributeError: 'SVC' object has no attribute 'SVC'

In [27]:
###gridsearch using rf

tuned_parameters = [{'C':[1,5,10,20]}]   
    
scores = ['r2']

for score in scores:
    
    print score
    
    clf = GridSearchCV(svm.SVC(kernel=str('linear'),class_weight='balanced'), tuned_parameters, cv=5)
    clf.fit(ngram_content, np.array(category_id_list))

    print("别！喝！咖！啡！了！最佳参数找到了亲！！：")
    print ""
    #best_estimator_ returns the best estimator chosen by the search
    print(clf.best_estimator_)
    print ""
    print("得分分别是:")
    print ""
    #grid_scores_的返回值:
    #    * a dict of parameter settings
    #    * the mean score over the cross-validation folds 
    #    * the list of scores for each fold
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print ""


r2
别！喝！咖！啡！了！最佳参数找到了亲！！：

SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

得分分别是:

0.809 (+/-0.004) for {'C': 1}
0.808 (+/-0.009) for {'C': 5}
0.801 (+/-0.009) for {'C': 10}
0.804 (+/-0.010) for {'C': 20}



In [100]:
print(len(contentLines))

882


In [128]:
whole_words = []
print(contentLines[1].split(' '))
#for line in contentLines:
#    contentLineWords = line.strip()
    

[u'june', u'14', u',', u'smrt', u'adding', u'seven', u'new', u'wheelchair', u'accessible', u'bus', u'(', u'wab', u')', u'service', u',', u'thereby', u'increasing', u'wab', u'service', u'route', u'52', u'total', u'.', u'additional', u'bus', u'service', u'added', u'wab', u'service', u'106', u',', u'190', u',', u'301', u',', u'804', u',', u'911', u',', u'963', u'983', u'.']


In [129]:
import re
temp = "?!@forum writer mr han jiajing ( `` shorten interval time peak-hour train '' ; monday , mar 2 ) . argued fare increase given approval raised , one would expected train service improved . opined train service seem deteriorating instead seems attempt improve even basic . noticed cabin packed brim train arrive five six minute apart , even platform get overcrowded , result , travelling older train east-west line uncomfortable , especially air-conditioning working cabin . opined overcrowded condition safety concern well . free travel peak hour introduced , smrt assured commuter train would arrive two three minute apart , argued scenario materialise . mr han contended although land transport authority promised improvement , would take another two year happen . meantime , least smrt could ensure train arrival time improved air-conditioning older train maintained comfortable level . concluded train fare increase would justifiable improvement ."
temp = temp.decode("utf8")
string = re.sub("[\.,\(\);:\?!@]\'","",temp)
print string


?!@forum writer mr han jiajing ( `` shorten interval time peak-hour train '' ; monday , mar 2 ) . argued fare increase given approval raised , one would expected train service improved . opined train service seem deteriorating instead seems attempt improve even basic . noticed cabin packed brim train arrive five six minute apart , even platform get overcrowded , result , travelling older train east-west line uncomfortable , especially air-conditioning working cabin . opined overcrowded condition safety concern well . free travel peak hour introduced , smrt assured commuter train would arrive two three minute apart , argued scenario materialise . mr han contended although land transport authority promised improvement , would take another two year happen . meantime , least smrt could ensure train arrival time improved air-conditioning older train maintained comfortable level . concluded train fare increase would justifiable improvement .


In [130]:
##print(contentLines[3])
for i in range(len(contentLines)):
    contentLines[i] = re.sub("[\.,\(\);:\?!@\'\`]","",contentLines[i])
print(contentLines[90])

wanbao was active difficulty walking   madam ting lan kin said forgiven driver still family take care  according strait time report  smrt apologised madam ting s family suspended bus driver  smrt s vice-president corporate information communication patrick nathan said bus captain reminded put safety commuter first 


In [131]:
print(contentLines[3])
for line in contentLines:
    whole_words.extend(line.split())
    print(len(line.split()))
print(len(whole_words))

st car coe premium four-month high 
28
31
65
6
63
84
7
6
128
39
49
17
9
7
42
48
30
35
61
9
18
9
56
75
48
13
15
23
57
44
29
36
21
67
46
46
43
40
90
129
36
47
49
59
13
8
52
11
6
40
23
15
40
32
66
50
8
38
5
10
7
12
34
7
45
42
62
39
107
61
69
17
6
63
8
42
67
38
43
26
56
23
52
35
44
45
45
54
32
55
45
112
21
18
6
56
49
39
56
47
40
88
65
47
55
51
7
11
67
13
7
63
55
91
92
41
90
53
97
29
93
36
5
7
39
50
9
31
55
7
54
13
51
63
46
86
57
5
39
71
40
10
55
60
33
64
21
37
42
59
84
50
37
62
55
36
7
34
48
50
43
31
69
6
41
56
38
42
7
40
83
63
56
67
65
20
40
39
12
115
39
24
128
48
36
54
51
39
6
50
67
36
65
45
7
38
34
44
68
39
38
47
56
9
52
23
32
42
28
70
7
49
5
63
37
17
41
47
67
16
12
65
28
126
8
37
56
23
54
7
7
31
5
72
37
30
48
75
40
48
7
6
36
52
38
84
32
35
77
12
98
49
85
9
27
39
69
105
76
59
24
10
21
40
24
6
54
15
8
52
79
93
7
49
43
12
57
31
49
56
8
7
47
45
70
8
48
52
31
11
96
39
38
34
53
46
39
51
55
60
45
74
54
71
73
44
60
43
49
8
30
66
7
53
70
19
7
53
48
96
62
113
75
22
53
48
54
42
48
31
67
70
43
7
5

In [132]:
whole_words = []
for line in contentLines:
    whole_words.extend(line.split())
    print(len(line.split()))
print(len(whole_words))

word_dict = set(whole_words)
print(len(word_dict))
print(contentLines[2:10])

28
31
65
6
63
84
7
6
128
39
49
17
9
7
42
48
30
35
61
9
18
9
56
75
48
13
15
23
57
44
29
36
21
67
46
46
43
40
90
129
36
47
49
59
13
8
52
11
6
40
23
15
40
32
66
50
8
38
5
10
7
12
34
7
45
42
62
39
107
61
69
17
6
63
8
42
67
38
43
26
56
23
52
35
44
45
45
54
32
55
45
112
21
18
6
56
49
39
56
47
40
88
65
47
55
51
7
11
67
13
7
63
55
91
92
41
90
53
97
29
93
36
5
7
39
50
9
31
55
7
54
13
51
63
46
86
57
5
39
71
40
10
55
60
33
64
21
37
42
59
84
50
37
62
55
36
7
34
48
50
43
31
69
6
41
56
38
42
7
40
83
63
56
67
65
20
40
39
12
115
39
24
128
48
36
54
51
39
6
50
67
36
65
45
7
38
34
44
68
39
38
47
56
9
52
23
32
42
28
70
7
49
5
63
37
17
41
47
67
16
12
65
28
126
8
37
56
23
54
7
7
31
5
72
37
30
48
75
40
48
7
6
36
52
38
84
32
35
77
12
98
49
85
9
27
39
69
105
76
59
24
10
21
40
24
6
54
15
8
52
79
93
7
49
43
12
57
31
49
56
8
7
47
45
70
8
48
52
31
11
96
39
38
34
53
46
39
51
55
60
45
74
54
71
73
44
60
43
49
8
30
66
7
53
70
19
7
53
48
96
62
113
75
22
53
48
54
42
48
31
67
70
43
7
50
66
5
46
80
37
58
11
87
47
47
58
44

In [133]:
word_2_int = { k:v for k, v in zip(word_dict,range(1,len(word_dict)+1))  }

In [134]:
print(len(word_dict))

5305


In [135]:
print(len(range(len(word_dict))))

5305


In [136]:
print(len(word_2_int))

5305


In [137]:
print(min(word_2_int.values()))

1


In [138]:
print(word_2_int['car'])

4039


In [139]:
print(word_2_int)

{u'limited': 1, u'clean-up': 3, u'folklore': 3550, u'raining': 5, 'yellow': 7, u'hitch': 8, u'four': 9, u'catch': 10, u'asian': 11, u'6pm': 13, u'redacted': 3586, u'captain': 14, 'integrity': 15, u'worked': 16, u'localized': 17, u'whose': 124, u'voter': 19, u'981': 5068, u'tamping': 20, u'aug': 21, u'four-month-old': 22, u'electricity': 23, u'tweet': 24, u'disability': 25, u'seriously': 188, u'unanswered': 27, u'supported': 4917, u'caned': 29, u'voted': 30, u'620pm': 4481, u'1223pm': 33, u'stark': 4638, u'worth': 34, u'updated': 36, u'collaborate': 37, u'risk': 38, u'retail': 4464, u'dollar': 4633, u'void': 40, u'regional': 41, u'lijuan': 43, u'every': 45, u'car-free': 46, u'hotline': 5156, u'kick': 5019, u'affect': 47, u'month': 2680, u'bringing': 49, u'excited': 3690, u'encounter': 50, u'look': 3560, u'amran': 53, u'school': 54, u'molesting': 55, u'motorcyclist': 56, u'demerit': 57, u'good': 3030, u'960': 3559, u'venue': 58, u'wednesday': 59, u'29-year-old': 61, u'solid': 3563, u'con

In [140]:
def word2Int(wd):
    try:
        return(word_2_int[wd])
    except KeyError:
        return 0

print(word2Int(u'nuk'), word2Int(u'train'))

(0, 3961)


In [141]:
print(word2Int(u'nuk'), word2Int(u'train'))

(0, 3961)


In [142]:
def categoryInt2Array(idx,dim):
    category_list = [0]*dim
    category_list[idx] = 1
    return category_list

In [143]:
print(categoryInt2Array(2,10))

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [144]:
train_list = []
for line in contentLines:
    line_split = line.split()
    train_list.append(map(lambda x:word2Int(x),line_split))

In [145]:
print(len(train_list))
print(train_list[1])

817
[5278, 1915, 5287, 1002, 898, 113, 1653, 5059, 5192, 1186, 503, 1715, 115, 1186, 503, 4864, 1058, 174, 5169, 5192, 503, 3280, 1186, 503, 137, 2758, 3772, 629, 2473, 3562, 5066]


In [148]:
maxlen = 100
from keras.preprocessing import sequence
train_list = sequence.pad_sequences(train_list, maxlen=maxlen) ###padding
print(train_list[10:20])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0  113 5287 5192  503 1117
  2496 4170 1758 3000 3103 2496 4170 1758  619  610  915 3855 2668 4165
  1612 1628 1914 4701  776  711 3976 4818 5287 3951 5082 5069 4458  738
   503 2544 2496 4170 1758 5192 1860  113 3577  503 3767  323 1323 4309
  2785 4339]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0 3681
  1370 1115 2865  978  943 4190 3527  230 2630 2006

In [153]:
print(len(category_id_list))
print(len(train_list))
print(category_id_list[0])
train_y_list = map(lambda x: categoryInt2Array(x,5), category_id_list)
print(len(train_y_list))
print(train_y_list[1])

817
817
2
817
[1, 0, 0, 0, 0]


In [342]:
train_y_list1 = map(lambda x: categoryInt2Array(x,5), y_train)

In [343]:
train_y_list1

[[0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 1,

In [336]:

from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
model = Sequential()
model.add(Embedding(len(word_dict)+1, 64, input_length = maxlen))
model.add(LSTM(256,return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
#model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")

#model = Sequential()
#model.add(Dense(512, input_shape=(200,), activation='tanh'))

#model.add(Dropout(0.5))
#model.add(Dense(10))
#model.add(Activation('softmax'))
#model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', class_mode="categorical")


In [337]:
help(Embedding)

Help on class Embedding in module keras.layers.embeddings:

class Embedding(keras.engine.topology.Layer)
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
 |  
 |  This layer can only be used as the first layer in a model.
 |  
 |  # Example
 |  
 |  ```python
 |    model = Sequential()
 |    model.add(Embedding(1000, 64, input_length=10))
 |    # the model will take as input an integer matrix of size (batch, input_length).
 |    # the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
 |    # now model.output_shape == (None, 10, 64), where None is the batch dimension.
 |  
 |    input_array = np.random.randint(1000, size=(32, 10))
 |  
 |    model.compile('rmsprop', 'mse')
 |    output_array = model.predict(input_array)
 |    assert output_array.shape == (32, 10, 64)
 |  ```
 |  
 |  # Arguments
 |    input_dim: int > 0. Size of the vocabulary,
 |        i.e. maximum int

In [344]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', class_mode="categorical")
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, 100, 64)           339584    
_________________________________________________________________
lstm_81 (LSTM)               (None, 100, 256)          328704    
_________________________________________________________________
dropout_81 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
lstm_82 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dropout_82 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 5)                 645       
_________________________________________________________________
activation_41 (Activation)   (None, 5)                 0         
Total para

In [340]:
#from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test=train_test_split(train_list,np.array(train_y_list),test_size=0.1)                      

print(len(X_train),len(y_train))
print(len(X_test),len(y_test))
y_train[0]
X_train[0]

(735, 735)
(82, 82)


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 2668, 1612, 1777, 1359, 4615,
       5192, 4864, 1720, 1653, 5059, 5192, 1186,  503, 5192, 2528, 5287,
       1642, 3623, 4615, 5192,  503, 3038, 3773,  974,  487, 2810, 2475,
       2474, 3559, 3561,  113, 3173,  401, 5287,  263,  174, 4296, 1186,
        503], dtype=int32)

In [345]:
import numpy as np

#model.fit(np.array(X_train),np.array(y_train), validation_data = (X_test,y_test),epochs = 20, batch_size=32)
model.fit(np.array(X_train),np.array(train_y_list1),epochs = 20, batch_size=32)

Epoch 1/20
735/735 [==============================] - 41s - loss: 1.4779 - acc: 0.3578    
Epoch 2/20
735/735 [==============================] - 35s - loss: 1.2873 - acc: 0.4789    
Epoch 3/20
735/735 [==============================] - 34s - loss: 0.9143 - acc: 0.6395    
Epoch 4/20
735/735 [==============================] - 33s - loss: 0.6429 - acc: 0.7660    
Epoch 5/20
735/735 [==============================] - 33s - loss: 0.3614 - acc: 0.8844    
Epoch 6/20
735/735 [==============================] - 33s - loss: 0.3068 - acc: 0.8884    
Epoch 7/20
735/735 [==============================] - 33s - loss: 0.1856 - acc: 0.9469    
Epoch 8/20
735/735 [==============================] - 33s - loss: 0.1396 - acc: 0.9660    
Epoch 9/20
735/735 [==============================] - 33s - loss: 0.1165 - acc: 0.9687    
Epoch 10/20
735/735 [==============================] - 34s - loss: 0.0687 - acc: 0.9782    
Epoch 11/20
735/735 [==============================] - 34s - loss: 0.0760 - acc: 0.9810  

In [ ]:
Train on 735 samples, validate on 82 samples
Epoch 1/20
735/735 [==============================] - 37s - loss: 1.5801 - acc: 0.3878 - val_loss: 1.3545 - val_acc: 0.4756
Epoch 2/20
735/735 [==============================] - 34s - loss: 1.2721 - acc: 0.5224 - val_loss: 1.2848 - val_acc: 0.5366
Epoch 3/20
735/735 [==============================] - 33s - loss: 0.9151 - acc: 0.6667 - val_loss: 1.1293 - val_acc: 0.5366
Epoch 4/20
735/735 [==============================] - 35s - loss: 0.7120 - acc: 0.7429 - val_loss: 1.0190 - val_acc: 0.6829
Epoch 5/20
735/735 [==============================] - 39s - loss: 0.5435 - acc: 0.8354 - val_loss: 0.9393 - val_acc: 0.6585
Epoch 6/20
735/735 [==============================] - 38s - loss: 0.2892 - acc: 0.9048 - val_loss: 0.8968 - val_acc: 0.6829
Epoch 7/20
735/735 [==============================] - 40s - loss: 0.2038 - acc: 0.9279 - val_loss: 1.0836 - val_acc: 0.6707
Epoch 8/20
735/735 [==============================] - 41s - loss: 0.2205 - acc: 0.9333 - val_loss: 0.9603 - val_acc: 0.6707
Epoch 9/20
735/735 [==============================] - 39s - loss: 0.0918 - acc: 0.9782 - val_loss: 1.0582 - val_acc: 0.7195
Epoch 10/20
735/735 [==============================] - 38s - loss: 0.1336 - acc: 0.9592 - val_loss: 1.0861 - val_acc: 0.6707
Epoch 11/20
735/735 [==============================] - 37s - loss: 0.0577 - acc: 0.9864 - val_loss: 1.0447 - val_acc: 0.7317
Epoch 12/20
735/735 [==============================] - 34s - loss: 0.0766 - acc: 0.9796 - val_loss: 1.0014 - val_acc: 0.7317
Epoch 13/20
735/735 [==============================] - 37s - loss: 0.1761 - acc: 0.9633 - val_loss: 1.1420 - val_acc: 0.7317
Epoch 14/20
735/735 [==============================] - 37s - loss: 0.0164 - acc: 0.9973 - val_loss: 1.2075 - val_acc: 0.7561
Epoch 15/20
735/735 [==============================] - 40s - loss: 0.0262 - acc: 0.9932 - val_loss: 1.1688 - val_acc: 0.7439
Epoch 16/20
735/735 [==============================] - 43s - loss: 0.0129 - acc: 0.9973 - val_loss: 1.4270 - val_acc: 0.7317
Epoch 17/20
735/735 [==============================] - 39s - loss: 0.0494 - acc: 0.9878 - val_loss: 1.4529 - val_acc: 0.7073
Epoch 18/20
735/735 [==============================] - 39s - loss: 0.0520 - acc: 0.9918 - val_loss: 1.4088 - val_acc: 0.7195
Epoch 19/20
735/735 [==============================] - 40s - loss: 0.0542 - acc: 0.9850 - val_loss: 1.3324 - val_acc: 0.7317
Epoch 20/20
735/735 [==============================] - 40s - loss: 0.0071 - acc: 0.9973 - val_loss: 1.5658 - val_acc: 0.7317


In [348]:
y_test1

array([1, 2, 2, 2, 1, 4, 1, 1, 1, 2, 4, 2, 4, 1, 2, 1, 2, 2, 4, 2, 2, 1, 2,
       1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 1, 2,
       1, 1, 2, 2, 1, 1, 1, 1, 2, 4, 1, 3, 2, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 2, 4, 2, 2, 1, 2, 1])

In [349]:
y_test_2

array([2, 1, 2, 1, 1, 4, 2, 3, 1, 1, 2, 2, 1, 4, 1, 1, 3, 1, 0, 2, 1, 2, 2,
       1, 1, 1, 2, 2, 1, 0, 2, 4, 2, 1, 4, 2, 2, 2, 1, 0, 4, 2, 3, 1, 4, 2,
       1, 2, 3, 2, 4, 4, 1, 4, 1, 1, 2, 2, 1, 1, 2, 0, 1, 1, 1, 2, 2, 1, 2,
       2, 1, 1, 1, 1, 4, 2, 2, 4, 4, 1, 1, 1])

In [350]:
y_test

array([1, 2, 2, 0, 1, 3, 1, 2, 1, 2, 4, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       1, 2, 2, 1, 1, 1, 1, 1, 2, 4, 1, 1, 1, 4, 1, 4, 4, 4, 1, 1, 4, 1, 0,
       1, 1, 4, 2, 1, 1, 1, 1, 0, 4, 1, 4, 2, 1, 1, 0, 4, 2, 1, 2, 1, 1, 1,
       4, 2, 2, 1, 2, 1, 2, 4, 2, 2, 1, 2, 1])

In [351]:
y_test1 = model.predict(X_test).argmax(axis= 1)
##y_test_2 = [i.index(1) for i in y_test.tolist()]
y_test = np.array(y_test)
print(y_test_2[y_test_2==1])
print(y_test1[y_test_2==1])
from sklearn.metrics import precision_score, recall_score, f1_score
p = precision_score(y_test_2[y_test_2==1], y_test1[y_test_2==1],average='weighted')
for i in range(6):
    p = precision_score(y_test_2[y_test_2==i], y_test1[y_test_2==i],average='weighted')
    print(" p is {0} ".format(p))
r = recall_score(y_test, y_test1,average='weighted')
f1score = f1_score(y_test, y_test1,average='weighted')
p = precision_score(y_test, y_test1,average='weighted') 

print(r)
print(f1score)
print(p)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2 2 1 1 2 4 2 1 2 2 1 2 2 1 1 3 2 1 1 2 4 2 1 1 1 1 1 1 1 1 4 1 2 1]
 p is 1.0 
 p is 1.0 
 p is 1.0 
 p is 0.0 
 p is 1.0 
 p is 0 
0.731707317073
0.707625030796
0.721095977194


In [ ]:
import numpy as np
from sklearn import metrics
y = np.array(y_test_2)
pred = np.array(y_test1)
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
score = model.evaluate(X_test,y_test)

In [123]:
help(model.evaluate)
print(score)

Help on method evaluate in module keras.models:

evaluate(self, x, y, batch_size=32, verbose=1, sample_weight=None) method of keras.models.Sequential instance
    Computes the loss on some input data, batch by batch.
    
    # Arguments
        x: input data, as a Numpy array or list of Numpy arrays
            (if the model has multiple inputs).
        y: labels, as a Numpy array.
        batch_size: integer. Number of samples per gradient update.
        verbose: verbosity mode, 0 or 1.
        sample_weight: sample weights, as a Numpy array.
    
    # Returns
        Scalar test loss (if the model has no metrics)
        or list of scalars (if the model computes other metrics).
        The attribute `model.metrics_names` will give you
        the display labels for the scalar outputs.
    
    # Raises
        RuntimeError: if the model was never compiled.

[2.3179928205781062, 0.7175141263142818]


In [169]:
contentLines[10]

u'new smrt bus service ply choa chu kang avenue 7  choa chu kang grove keat hong link starting next sunday  may 15   joint news release issued tuesday  smrt land transport authority  lta  said service start choa chu kang bus interchange  new loop service 308 operate daily  including public holiday '

In [ ]:
{u'service announcements': 40,
 u'crisis / delays & distributions / accidents / safety': 323,
 u'facilities & services': 264, 
 u'service excellence': 37, 
 u'general / others': 153}

In [358]:
contentLines_np = np.array(contentLines)
contentLines_np[y_test_2 == 4]

/Users/fay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 817 but corresponding boolean dimension is 82
  from ipykernel import kernelapp as app


array([ u'taxi operator taking fight uber grab setting private-hire limousine arm  aim retain driver begun defect fast-expanding private-hire fleet run third-party taxi app provider  smrt corp ordered fleet 300 new toyota vellfire mpvs  corolla altis sedan lexus car purpose  taxi industry player noted unlike cab company  newcomer required meet service standard  fleet expansion restricted  need buy insurance policy cover passenger  restriction vehicle age  car attract far lower road tax cab  others said fast-expanding third-party fleet eventually impact cabby  earnings competing fixed market growing nearly fast ',
       u'st video point u duo train vandal ',
       u'fuelled aggressive bid uber  certificate entitlement  coes  car ended higher latest tender yesterday  despite prospect bigger quota two week  industry watcher said upward pressure bid may also come private-hire company grab smrt s new outfit  stride ',
       u'reader  ronnie lim ah bee  questioned possible smrt look  hybr

In [177]:
categort2id_dict

{u'crisis / delays & distributions / accidents / safety': 1,
 u'facilities & services': 2,
 u'general / others': 4,
 u'service announcements': 0,
 u'service excellence': 3}

In [182]:
category_id_list[0:10]

[2, 0, 1, 2, 1, 4, 2, 2, 0, 2]

In [283]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_list,np.array(category_id_list),test_size=0.1)

In [284]:
category_id_list_0 =  [1 if id ==0 else 0 for id in y_train ]
category_id_list_1 =  [1 if id ==1 else 0 for id in y_train ]
category_id_list_2 =  [1 if id ==2 else 0 for id in y_train ]
category_id_list_3 =  [1 if id ==3 else 0 for id in y_train ]
category_id_list_4 =  [1 if id ==4 else 0 for id in y_train ]

In [285]:
category_id_list_2[0:10]

[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]

In [286]:
train_y_list_0 = map(lambda x: categoryInt2Array(x,2), category_id_list_0)
print(np.array(train_y_list_0).shape)
print(np.array(X_train).shape)

(735, 2)
(735, 100)


In [290]:

from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
model0 = Sequential()
model0.add(Embedding(len(word_dict)+1, 64, input_length = maxlen))
model0.add(LSTM(256,return_sequences = True))
model0.add(Dropout(0.5))
model0.add(LSTM(128, return_sequences = False))
model0.add(Dropout(0.5))
model0.add(Dense(1))
model0.add(Activation('sigmoid'))
model0.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(len(X_train),len(y_train))
print(len(train_y_list_0),len(train_y_list_0))

import numpy as np

model0.fit(np.array(X_train),category_id_list_0, epochs = 10, batch_size=32)

(735, 735)
(735, 735)
Epoch 1/10
735/735 [==============================] - 38s - loss: 0.2970 - acc: 0.9347    
Epoch 2/10
735/735 [==============================] - 32s - loss: 0.1636 - acc: 0.9537    
Epoch 3/10
735/735 [==============================] - 32s - loss: 0.0783 - acc: 0.9769    
Epoch 4/10
735/735 [==============================] - 32s - loss: 0.0291 - acc: 0.9905    
Epoch 5/10
735/735 [==============================] - 32s - loss: 0.0115 - acc: 0.9946    
Epoch 6/10
735/735 [==============================] - 32s - loss: 0.0127 - acc: 0.9932    
Epoch 7/10
735/735 [==============================] - 32s - loss: 0.0629 - acc: 0.9850    
Epoch 8/10
735/735 [==============================] - 32s - loss: 0.0033 - acc: 1.0000    
Epoch 9/10
735/735 [==============================] - 33s - loss: 0.0014 - acc: 1.0000    
Epoch 10/10
735/735 [==============================] - 32s - loss: 3.5967e-04 - acc: 1.0000    


In [291]:
train_y_list_1 = map(lambda x: categoryInt2Array(x,2), category_id_list_1)
print(len(train_y_list_1))
print(train_y_list_1[1])

735
[1, 0]


In [293]:

from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
model1 = Sequential()
model1.add(Embedding(len(word_dict)+1, 64, input_length = maxlen))
model1.add(LSTM(256,return_sequences = True))
model1.add(Dropout(0.5))
model1.add(LSTM(128, return_sequences = False))
model1.add(Dropout(0.5))
model1.add(Dense(1))
model1.add(Activation('sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


import numpy as np

model1.fit(np.array(X_train),category_id_list_1,epochs = 10, batch_size=32)

Epoch 1/10
735/735 [==============================] - 39s - loss: 0.7062 - acc: 0.6000    
Epoch 2/10
735/735 [==============================] - 32s - loss: 0.3554 - acc: 0.8422    
Epoch 3/10
735/735 [==============================] - 33s - loss: 0.1848 - acc: 0.9374    
Epoch 4/10
735/735 [==============================] - 32s - loss: 0.8166 - acc: 0.7456    
Epoch 5/10
735/735 [==============================] - 32s - loss: 0.6791 - acc: 0.5905    
Epoch 6/10
735/735 [==============================] - 32s - loss: 0.6824 - acc: 0.5905    
Epoch 7/10
735/735 [==============================] - 32s - loss: 0.6700 - acc: 0.6054    
Epoch 8/10
735/735 [==============================] - 32s - loss: 0.6678 - acc: 0.5986    
Epoch 9/10
735/735 [==============================] - 32s - loss: 0.5210 - acc: 0.7252    
Epoch 10/10
735/735 [==============================] - 33s - loss: 0.1089 - acc: 0.9660    


In [294]:
train_y_list_2 = map(lambda x: categoryInt2Array(x,2), category_id_list_2)
print(len(train_y_list_2))
print(train_y_list_2[1])

735
[1, 0]


In [296]:

from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
model2 = Sequential()
model2.add(Embedding(len(word_dict)+1, 64, input_length = maxlen))
model2.add(LSTM(256,return_sequences = True))
model2.add(Dropout(0.5))
model2.add(LSTM(128, return_sequences = False))
model2.add(Dropout(0.5))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



model2.fit(np.array(X_train),category_id_list_2,epochs = 10, batch_size=32)

Epoch 1/10
735/735 [==============================] - 39s - loss: 0.6421 - acc: 0.6830    
Epoch 2/10
735/735 [==============================] - 32s - loss: 0.4405 - acc: 0.8204    
Epoch 3/10
735/735 [==============================] - 32s - loss: 0.2185 - acc: 0.9224    
Epoch 4/10
735/735 [==============================] - 32s - loss: 0.1285 - acc: 0.9551    
Epoch 5/10
735/735 [==============================] - 32s - loss: 0.0510 - acc: 0.9810    
Epoch 6/10
735/735 [==============================] - 32s - loss: 0.0536 - acc: 0.9864    
Epoch 7/10
735/735 [==============================] - 32s - loss: 0.0263 - acc: 0.9905    
Epoch 8/10
735/735 [==============================] - 32s - loss: 0.0411 - acc: 0.9891    
Epoch 9/10
735/735 [==============================] - 32s - loss: 0.0220 - acc: 0.9932    
Epoch 10/10
735/735 [==============================] - 32s - loss: 0.0124 - acc: 0.9986    


In [297]:
train_y_list_3 = map(lambda x: categoryInt2Array(x,2), category_id_list_3)
print(len(train_y_list_3))
print(train_y_list_3[1])

735
[1, 0]


In [299]:

from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
model3 = Sequential()
model3.add(Embedding(len(word_dict)+1, 64, input_length = maxlen))
model3.add(LSTM(256,return_sequences = True))
model3.add(Dropout(0.5))
model3.add(LSTM(128, return_sequences = False))
model3.add(Dropout(0.5))
model3.add(Dense(1))
model3.add(Activation('sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(len(X_train),len(y_train))
print(len(X_test),len(y_test))

import numpy as np

model3.fit(np.array(X_train),category_id_list_3, epochs = 10, batch_size=32)

(735, 735)
(82, 82)
Epoch 1/10
735/735 [==============================] - 40s - loss: 0.2937 - acc: 0.9279    
Epoch 2/10
735/735 [==============================] - 32s - loss: 0.2072 - acc: 0.9510    
Epoch 3/10
735/735 [==============================] - 32s - loss: 0.2151 - acc: 0.9510    
Epoch 4/10
735/735 [==============================] - 32s - loss: 0.2059 - acc: 0.9510    
Epoch 5/10
735/735 [==============================] - 32s - loss: 0.1993 - acc: 0.9510    
Epoch 6/10
735/735 [==============================] - 32s - loss: 0.2026 - acc: 0.9510    
Epoch 7/10
735/735 [==============================] - 32s - loss: 0.2064 - acc: 0.9510    
Epoch 8/10
735/735 [==============================] - 32s - loss: 0.2041 - acc: 0.9510    
Epoch 9/10
735/735 [==============================] - 32s - loss: 0.0857 - acc: 0.9714    
Epoch 10/10
735/735 [==============================] - 32s - loss: 0.0326 - acc: 0.9918    


In [300]:
train_y_list_4 = map(lambda x: categoryInt2Array(x,2), category_id_list_4)
print(len(train_y_list_4))
print(train_y_list_4[1])

735
[0, 1]


In [301]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
model4 = Sequential()
model4.add(Embedding(len(word_dict)+1, 64, input_length = maxlen))
model4.add(LSTM(256,return_sequences = True))
model4.add(Dropout(0.5))
model4.add(LSTM(128, return_sequences = False))
model4.add(Dropout(0.5))
model4.add(Dense(1))
model4.add(Activation('sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model4.fit(np.array(X_train),category_id_list_4,epochs = 10, batch_size=32)

Epoch 1/10
735/735 [==============================] - 39s - loss: 0.5845 - acc: 0.7905    
Epoch 2/10
735/735 [==============================] - 32s - loss: 0.5112 - acc: 0.8109    
Epoch 3/10
735/735 [==============================] - 32s - loss: 0.5085 - acc: 0.8095    
Epoch 4/10
735/735 [==============================] - 33s - loss: 0.4222 - acc: 0.8313    
Epoch 5/10
735/735 [==============================] - 32s - loss: 0.1841 - acc: 0.9374    
Epoch 6/10
735/735 [==============================] - 32s - loss: 0.0800 - acc: 0.9741    
Epoch 7/10
735/735 [==============================] - 32s - loss: 0.0418 - acc: 0.9878    
Epoch 8/10
735/735 [==============================] - 33s - loss: 0.0239 - acc: 0.9918    
Epoch 9/10
735/735 [==============================] - 32s - loss: 0.0334 - acc: 0.9946    
Epoch 10/10
735/735 [==============================] - 33s - loss: 0.0123 - acc: 0.9959    


In [303]:
model1_result = model1.predict(X_test)[:,0]
model2_result = model2.predict(X_test)[:,0]
model3_result = model3.predict(X_test)[:,0]
model4_result = model4.predict(X_test)[:,0]
model0_result = model0.predict(X_test)[:,0]

In [304]:
model1_result  = model1_result.reshape(-1,1)
model2_result = model2_result.reshape(-1,1)
model3_result = model3_result.reshape(-1,1)
model4_result = model4_result.reshape(-1,1)
model0_result = model0_result.reshape(-1,1)

In [305]:
model0_result

array([[  2.86768809e-05],
       [  3.33532807e-05],
       [  6.82218059e-04],
       [  4.77994414e-04],
       [  1.51940685e-05],
       [  1.22510937e-05],
       [  1.15112498e-05],
       [  2.43973700e-05],
       [  1.54934896e-05],
       [  5.46581441e-06],
       [  1.22016900e-05],
       [  4.78315997e-06],
       [  1.41219971e-05],
       [  9.27181645e-06],
       [  1.00743255e-05],
       [  2.75061902e-05],
       [  2.51952242e-05],
       [  1.20358591e-05],
       [  1.58115781e-05],
       [  2.15519885e-05],
       [  9.36683227e-06],
       [  2.05300366e-05],
       [  6.84353336e-06],
       [  1.09391030e-05],
       [  2.49889799e-05],
       [  3.94418712e-05],
       [  5.20881758e-06],
       [  1.59212923e-05],
       [  1.12702965e-05],
       [  2.45980518e-05],
       [  1.45519552e-05],
       [  1.17674617e-05],
       [  2.06890982e-05],
       [  1.74981851e-05],
       [  1.18271682e-05],
       [  1.01286769e-05],
       [  1.42381859e-05],
 

In [319]:
y_test1 = np.concatenate((model0_result,model1_result, model2_result,model3_result,model4_result), axis=1).argmax(axis= 1)

In [320]:
y_test1

array([1, 2, 2, 2, 1, 3, 1, 1, 1, 2, 4, 2, 1, 1, 2, 1, 2, 2, 4, 2, 2, 1, 2,
       1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 4, 2, 4, 1, 4, 1, 1, 2, 1, 3,
       1, 1, 4, 2, 1, 1, 1, 4, 2, 4, 1, 4, 2, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 1, 4, 2, 2, 1, 2, 1])

In [321]:
y_test

array([1, 2, 2, 0, 1, 3, 1, 2, 1, 2, 4, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       1, 2, 2, 1, 1, 1, 1, 1, 2, 4, 1, 1, 1, 4, 1, 4, 4, 4, 1, 1, 4, 1, 0,
       1, 1, 4, 2, 1, 1, 1, 1, 0, 4, 1, 4, 2, 1, 1, 0, 4, 2, 1, 2, 1, 1, 1,
       4, 2, 2, 1, 2, 1, 2, 4, 2, 2, 1, 2, 1])

In [322]:

#y_test_2 = [i.index(1) for i in y_test.tolist()]
#y_test_2 = np.array(y_test_2)

#from sklearn.metrics import precision_score, recall_score, f1_score
#p = precision_score(y_test_2[y_test_2==1], y_test1[y_test_2==1],average='weighted')
#for i in range(6):
#    p = precision_score(y_test_2[y_test_2==i], y_test1[y_test_2==i],average='weighted')
#    print(" p is {0} ".format(p))
r = recall_score(y_test, y_test1,average='weighted')
f1score = f1_score(y_test, y_test1,average='weighted')
p = precision_score(y_test, y_test1,average='weighted') 

print(r)
print(f1score)
print(p)

0.768292682927
0.757472058381
0.779517738359


In [329]:
import numpy as np
from sklearn import metrics
y = np.array(y_test1)
pred = np.array(y_test)
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=0)
metrics.auc(fpr, tpr)

0.018518518518518545

In [318]:
y_test1

array([1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0])